In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this(by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
import torch
import torch.nn as N
from torch.optim import Adam
import torch.nn.functional as F
import torch.optim as op

# Any results you write to the current directory are saved as output.

In [ ]:
dframe = pd.read_csv("../input/entity-annotated-corpus/ner.csv", encoding = "ISO-8859-1", error_bad_lines=False)

In [ ]:
dframe.head()



In [ ]:
df1=dframe[['word','tag']]

In [ ]:
data=[]
label=[]
a=[]
b=[]
for x in zip(df1.word,df1.tag):
    
    if x[0]!=".":
        a.append(x[0])
        b.append(x[1])
    
    else:
        data.append(a)
        label.append(b)
        a=[]
        b=[]
        

In [ ]:
data1=data[:20000]
label1=label[:20000]

In [ ]:
maxlen=max(map(len,data1))

In [ ]:
words=set()
for i in data:
    words.update(i)

In [ ]:
word2idx=dict([(j,i+1) for i,j in enumerate(words)])
idx2word=dict([(j,i) for i,j in word2idx.items()])
idxdata=[[word2idx[j] for j in s] for s in data]
label1=set()
for i in label:
    label1.update(i)



In [ ]:
word2idx['<pad>']=0
idx2word[0]='<pad>'

In [ ]:
label2idx=dict([(i,j+1) for j,i in enumerate(label1)])
idx2label=dict([(i,j) for j,i in label2idx.items()])
idxlabel=[[label2idx[k] for k in s] for s in label]

In [ ]:
def generator(data,label,batchsize,nepochs):
    maxlen=max(map(len,data))
    nbatches=len(data)/batchsize
    data=data[:int(nbatches*batchsize)]
    label=label[:int(nbatches*batchsize)]
    for i in range(nepochs):
        idx=np.random.permutation(np.arange(nbatches*batchsize).astype(np.int))
        data=data[idx]
        label=label[idx]
        
        for j in range(int(nbatches)):
            batchx=np.zeros((batchsize,maxlen))
            batchy=np.zeros((batchsize,maxlen))
            x=data[j*batchsize:(j+1)*batchsize]
            y=label[j*batchsize:(j+1)*batchsize]
            for m in range(batchsize):
                batchx[m,:len(x[m])]=x[m]
                batchy[m,:len(y[m])]=y[m]
            
            yield batchx,batchy

In [ ]:
idxdata=np.array(idxdata)
idxlabel=np.array(idxlabel)

In [ ]:
class model(N.Module):
    def __init__(self,vocabsize,embeddingdim,hiddendim,outputdim):
        super().__init__()
        self.embedding=N.Embedding(vocabsize,embeddingdim)
        self.drop1=N.Dropout(0.2)
        self.lstm=N.LSTM(embeddingdim,hiddendim,num_layers=2)
        self.fc=N.Linear(hiddendim,outputdim)
        
    def forward(self,x):
        embeds=self.embedding(x)
        #print(embeds.shape)
        embeds=self.drop1(embeds.permute(1,0,2))
        out1=self.lstm(embeds)   # seqlen x batchsize x 
        #print(out1[0].shape)
        out=self.fc(out1[0].permute(1,0,2))
        #print(out.shape)
        return out
        
    

In [ ]:
label2idx['<pad>']=0

In [ ]:
idx2label[0]='<pad>'

In [ ]:
vocabsize=len(idx2word)
embeddim=164
hiddendim=256
outputdim=len(label2idx)

In [ ]:
model1=model(vocabsize,embeddim,hiddendim,outputdim)

In [ ]:
def count(model1):
    return sum([l.numel() for l in model1.parameters()])

In [ ]:
count(model1)

In [ ]:
optimizer=op.Adam(model1.parameters(),lr=0.0001)
loss=N.CrossEntropyLoss()

In [ ]:
nepochs=2
batchsize=100

In [ ]:
gen1=generator(idxdata,idxlabel,batchsize,nepochs)

In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [ ]:
model1=model1.to(device)
loss=loss.to(device)

In [ ]:
trainloss=[]
count=0
for i in range(nepochs):
    for j in range(int(len(idxdata)/batchsize)):
        batchx,batchy=next(gen1)
       
        batchx1=torch.from_numpy(batchx).long()
        batchy1=torch.from_numpy(batchy).long()
        #print(batchx1)
        optimizer.zero_grad()
        batchx1=batchx1.to(device)  # very important to move tensors on gpu on which computations are taking place
        batchy1=batchy1.to(device)  # we moved the model and all tensors for computation on gpu
        output=model1.forward(batchx1)
        
        print(output.shape)
        print(batchy1.shape)
        l=loss(output.view(-1,19),batchy1.view(-1,)) #this is really important actual values should be of  shape (N), while pred (N,c)
        l.backward()
        optimizer.step()
        print("loss is:-",l," and epoch is:-",i," count is ",j)
    